## Training 

This section contains implementation specifics of building a CNN based image classifier using the iNaturalist dataset.

The Architecture:
1.   Five convolution layers with each layer followed by a 
ReLU activation and a max pooling layer.
2.   One dense layer 
3.   One output layer containing 10 neurons (1 for each of the 10 classes). 

Import essential libraries

In [1]:
# Essentials
import numpy as np
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation

import random
import imageio
import os
import cv2
import glob
random.seed(42)

In [2]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 133kB 19.3MB/s 
     |████████████████████████████████| 163kB 16.7MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 


Fetch dataset from GitHub

In [3]:
# Fetch the dataset from Github
!git clone https://github.com/borate267/inaturalist-dataset.git

Cloning into 'inaturalist-dataset'...
remote: Enumerating objects: 12027, done.
remote: Total 12027 (delta 0), reused 0 (delta 0), pack-reused 12027
Receiving objects: 100% (12027/12027), 3.55 GiB | 50.24 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (11999/11999), done.


Read the training and validation images

In [4]:
# Define the labels for the Simpsons characters we're detecting
class_names = {0:'Amphibia', 1:'Animalia', 2:'Arachnida',3: 'Aves',4: 'Fungi',
              5: 'Insecta', 6:'Mammalia', 7:'Mollusca', 8:'Plantae',9: 'Reptilia'}
num_classes = 10
img_size = 128
dir = 'inaturalist-dataset/train'

import random

# Load training data
X_train = []
y_train = []
for label, name in class_names.items():
   list_images = os.listdir(dir+'/'+name)
   for image_name in list_images:
       image = imageio.imread(dir+'/'+name+'/'+image_name)
       if np.ndim(image) == 3:
          X_train.append(cv2.resize(image, (img_size,img_size)))
          y_train.append(label)


Shuffle the images and then retain 10% as validation data

In [5]:
leng = np.shape(X_train)
arr = np.arange(leng[0])
np.random.shuffle(arr)
X_train_shuf = []
y_train_shuf = []
X_val_shuf = []
y_val_shuf = []

for i in range(leng[0]):
  if i <= 9000:
    X_train_shuf.append(X_train[arr[i]])
    y_train_shuf.append(y_train[arr[i]])
  else:
    X_val_shuf.append(X_train[arr[i]])
    y_val_shuf.append(y_train[arr[i]])

X_train = np.array(X_train_shuf)
y_train = np.array(y_train_shuf)
X_val = np.array(X_val_shuf)
y_val = np.array(y_val_shuf)

# Normalize the data
X_train = X_train/255.0
X_val = X_val/255.0

# One hot encode the labels 
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)


Configure the sweep hyperparameter dictionary

In [6]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[(3,3),(3,3),(3,3),(3,3),(3,3)], [(3,3),(5,5),(5,5),(7,7),(7,7)], [(7,7),(7,7),(5,5),(5,5),(3,3)], [(3,3),(5,5),(7,7),(9,9),(11,11)] ]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': ['true','false']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'num_dense':{
            'values': [64, 128, 256, 512]
        }
    }
}

 Initialize the Sweep

In [7]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity="bharatik", project="cs6910assignment2")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 72r3w5eh
Sweep URL: https://wandb.ai/bharatik/cs6910assignment2/sweeps/72r3w5eh


In [8]:
def train():
    
    config_defaults = {
        'kernel_size': [(3,3),(3,3),(3,3),(3,3),(3,3)],
        'weight_decay': 0.005,
        'dropout': 0.2,
        'learning_rate': 1e-3,
        'activation': 'relu',
        'batch_size': 64,
        'epochs': 10,
        'batch_norm': 'true',
        'filt_org' : [32,32,32,32,32],
        'conv_layer_size' : 16,
        'data_augment': 'true',
        'num_dense': 256,
        'seed': 42,
        'num_classes': 10
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    wandb.run.name = 'num_dense_'+ str(config.num_dense)+'_bs_'+str(config.batch_size)+'_ac_'+ config.activation
    
    # Determine input shape
    input_shape = (img_size, img_size , 3)
    
    # Define the model architecture
    model = Sequential()

    filter = config.filt_org

    # Layer one
    model.add(Conv2D(filters = filter[0], kernel_size = config.kernel_size[0],padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

    if config.batch_norm == 'True':
        model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    # Layer two
    model.add(Conv2D(filters = filter[1], kernel_size = config.kernel_size[1], padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

    if config.batch_norm == 'True':
        model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    # Layer three
    model.add(Conv2D(filters = filter[2], kernel_size = config.kernel_size[2], padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

    if config.batch_norm == 'True':
        model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer four
    model.add(Conv2D(filters = filter[3], kernel_size = config.kernel_size[3], padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

    if config.batch_norm == 'True':
        model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    

    # Layer five
    model.add(Conv2D(filters = filter[4], kernel_size = config.kernel_size[4], padding = 'same', 
                    input_shape = input_shape, kernel_regularizer=regularizers.l2(config.weight_decay)))

    if config.activation == "relu":
        model.add(Activation('relu'))
    elif config.activation == "elu":
        model.add(Activation('elu'))
    elif config.activation == "selu":
        model.add(Activation('selu'))

    if config.batch_norm == 'True':
        model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    # FC layer
    model.add(Flatten())
    model.add(Dense(config.num_dense, activation = config.activation, kernel_regularizer = regularizers.l2(config.weight_decay)))
    model.add(Dropout(config.dropout))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation = "softmax"))

    # Define the optimizer
    optimizer = Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999)
    
    model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

    #data augmentation
    if config.data_augment == 'true':
        datagen = ImageDataGenerator(
            rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False  # randomly flip images
        )
    else:
        datagen = ImageDataGenerator(rescale = 1.0)

    datagen.fit(X_train)
    
    model.fit(
        datagen.flow(X_train, y_train, batch_size = config.batch_size),
        epochs = config.epochs,
        verbose = 1,
        validation_data= (X_val, y_val),
        callbacks = [WandbCallback()]
    )
    

    
    

Run the sweep agent for 100 runs or more

In [9]:
# Initialize a new sweep

wandb.agent(sweep_id, train, count = 20)

wandb: Agent Starting Run: e7pfmxe9 with config:
wandb: 	activation: relu
wandb: 	batch_norm: true
wandb: 	batch_size: 64
wandb: 	data_augment: true
wandb: 	dropout: 0.4
wandb: 	filt_org: [128, 128, 64, 64, 32]
wandb: 	kernel_size: [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]
wandb: 	learning_rate: 0.0001
wandb: 	num_dense: 256
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: bharatik (use `wandb login --relogin` to force relogin)


Epoch 1/10
 58/141 [===========>..................] - ETA: 22s - loss: 2.6902 - accuracy: 0.1114

wandb: Ctrl + C detected. Stopping sweep.
